In [1]:
import numpy as np

In [2]:
def get_model(save_name='ResNet50.hdh5', image_shape=(224, 224, 3)):
    import os
    if not os.path.exists(save_name):
        from keras.applications.resnet50 import ResNet50
        model = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=image_shape, pooling='max')
        model.save(save_name)
    else:
        from keras.models import load_model
        model = load_model(save_name)
    return model

In [3]:
initial_model = get_model()

Using TensorFlow backend.
/usr/lib/python3.6/site-packages/keras/models.py:245: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def get_data(image_shape=(224, 224, 3)):
    from skimage.transform import resize
    from skimage.io import imread
    import os
    images = []
    labels = []
    with open('public_data/00_input/train/gt.csv') as navigate_file:
        for i, line in enumerate(navigate_file):
            if i == 0:
                continue
            image_name, label = line.split(',')                
            images.append(resize(
                imread(os.path.join(
                    'public_data/00_input/train/images/',
                    image_name
                )),
                image_shape
            ))
            labels.append(int(label))
    return np.array(images), np.array(labels)

In [5]:
x, y = get_data()

/usr/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [6]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [7]:
num_classes = len(set(y))
y = np.eye(num_classes)[y]
train_x, test_x, train_y, test_y = train_test_split(
    x, y, test_size=0.3, random_state=42
)

In [8]:
data_generator = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=True
)

In [9]:
last = initial_model.output
prediction = Dense(num_classes, activation='softmax')(last)
model = Model(initial_model.input, prediction)

In [10]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=0.0001),
    metrics=['categorical_accuracy']
)

In [11]:
batch_size = 16
model.fit_generator(
    data_generator.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=int(len(train_y) / batch_size), epochs=300,
    validation_data=(test_x, test_y)
)

Epoch 1/300
109/109 [==============================] - 82s - loss: 2.8655 - categorical_accuracy: 0.2638 - val_loss: 4.5544 - val_categorical_accuracy: 0.0147
Epoch 2/300
109/109 [==============================] - 74s - loss: 1.2118 - categorical_accuracy: 0.6527 - val_loss: 4.2262 - val_categorical_accuracy: 0.0120
Epoch 3/300
109/109 [==============================] - 74s - loss: 0.7254 - categorical_accuracy: 0.8035 - val_loss: 4.2945 - val_categorical_accuracy: 0.0173
Epoch 4/300
109/109 [==============================] - 74s - loss: 0.5106 - categorical_accuracy: 0.8555 - val_loss: 4.0729 - val_categorical_accuracy: 0.0320
Epoch 5/300
109/109 [==============================] - 74s - loss: 0.3599 - categorical_accuracy: 0.9019 - val_loss: 3.9999 - val_categorical_accuracy: 0.0640
Epoch 6/300
109/109 [==============================] - 74s - loss: 0.2641 - categorical_accuracy: 0.9274 - val_loss: 4.6184 - val_categorical_accuracy: 0.0800
Epoch 7/300
109/109 [=========================

KeyboardInterrupt: 

In [12]:
model.save('0.6_score_model.hdh5')